## Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# get modules
from executive_summary import *
#from cumulative_accounting import *
from air_quality import *
from fisheries import *
from noise import *
from recreation import *
from scenic_resources import *
from soil_conservation import *
from transportation_sustainablecommunities import *
from vegetation_preservation import *
from water_quality import *
from wildlife import *
localpath = pathlib.Path.cwd()

## Executive Summary

## Cumulative Accounting

In [ ]:
# development by district


In [ ]:
# get the data
dfDev19Bailey = pd.read_csv(localpath / 'data' / 'Summarize_ParcelLandCapabilities_Bailey_2019.csv')
sdf = dfDev19Bailey.copy()
# create new field for Land Capability Category as empty string
sdf['Category'] = ''
# set values to 'SEZ' if Majority_Land_Capab = '1B'
sdf.loc[sdf['Majority_CAPABILITY'].isin(['1B','WB']), 'Category'] = 'SEZ'
# set values to 'Sensitive' if Majority_Land_Capab = '1C', 1A, 2, 3
sdf.loc[sdf['Majority_CAPABILITY'].isin(['1C', '1A', '2', '3']), 'Category'] = 'Sensitive'
# set values to 'Non-Sensitive' if Majority_Land_Capab = '4', 5' or '6' 7
sdf.loc[sdf['Majority_CAPABILITY'].isin(['4', '5', '6', '7']), 'Category'] = 'Non-Sensitive'
# filter out WITHIN_TRPA_BOUNDARY = 0
sdf = sdf.loc[sdf['WITHIN_TRPA_BNDY'] == 1]
# melt by category and sum Residential_Units, TouristAccommodation_Units, and CommercialFloorArea_SqFt
df = pd.melt(sdf, id_vars=['Category'], value_vars=['Residential_Units', 'TouristAccommodation_Units', 'CommercialFloorArea_SqFt'], var_name='DevelopmentType', value_name='Value')
df = df.groupby(['Category', 'DevelopmentType'], dropna=False).sum().reset_index()
df.to_csv(localpath / 'data' / 'DevelopmentSummary_LandCap_TVAL2019.csv', index=False)

In [8]:
# summarize 2019 TVAL analysis results
dfOld19 = get_old_dev_cap()
sdf = dfOld19.copy()
# melt by category and sum Residential_Units, TouristAccommodation_Units, and CommercialFloorArea_SqFt
df = pd.melt(sdf, id_vars=['LandCapabilityType'], value_vars=['RES_2019', 'TAU_2019', 'CFA_2019'], var_name='DevelopmentType', value_name='Value')
df = df.groupby(['LandCapabilityType', 'DevelopmentType'], dropna=False).sum().reset_index()
# value to int
df['Value'] = df['Value'].astype(int)
df.to_csv(localpath / 'data' / 'DevelopmentSummary_LandCap_NEWDEVDATA2019.csv', index=False)

In [ ]:
# tables and charts for the cumulative accounting web page
# summarize existing development by land capability category
df23Bailey, sdf23Bailey = get_summary('2023')
df23Bailey.to_pickle(localpath / 'data' / 'df23Bailey.pkl')
sdf23Bailey.to_pickle(localpath / 'data' / 'sdf23Bailey.pkl')

In [ ]:
# tables and charts for the cumulative accounting web page
# summarize existing development by land capability category
df19Bailey, sdf19Bailey = get_summary('2019')
df19Bailey.to_pickle(localpath / 'data' / 'df19Bailey.pkl')
sdf19Bailey.to_pickle(localpath / 'data' / 'sdf19Bailey.pkl')

In [ ]:
sdfOld19 = df
df = pd.melt(sdfOld19, id_vars=['LandCapabilityType'], value_vars=['RES_2019', 'TAU_2019', 'CFA_2019'], var_name='DevelopmentType', value_name='Value')
df = df.groupby(['LandCapabilityType', 'DevelopmentType'], dropna=False).sum().reset_index()
# set Value to int
df['Value'] = df['Value'].astype(int)
dfOld19 = df
dfOld19.to_pickle(localpath / 'data' / 'dfOld19.pkl')

In [2]:
# write code to compare Units between dataframes using APN as the key
df19    = pd.read_pickle(localpath / 'data' / 'sdf19Bailey.pkl')
dfOld19 = pd.read_pickle(localpath / 'data' / 'sdfOld19.pkl')
df23    = pd.read_pickle(localpath / 'data' / 'sdf23Bailey.pkl')

In [ ]:
# sum dfOld19 Vlaue wher development type is TAU_2019
dfOld19Tau19 = dfOld19[dfOld19['DevelopmentType'] == 'CFA_2019']
dfOld19Tau19 = dfOld19Tau19.groupby(['LandCapabilityType'], dropna=False).sum().reset_index()
dfOld19Tau19

In [ ]:
# merge dataframes
df = pd.merge(df19, dfOld19, on='APN', how='outer', suffixes=('_2019', '_Old2019'))
# print rows where the values are different for LandCapability
df = df[df['Category'] != df['LandCapabilityType']]
df = df[['APN', 'Category', 'LandCapabilityType']]
df = df.dropna()
# show the value counts of each unique combination of Category and LandCapabilityType
df['Category'] = df['Category'].astype(str)
df['LandCapabilityType'] = df['LandCapabilityType'].astype(str)
df['Category_LandCapabilityType'] = df['Category'] + '_' + df['LandCapabilityType']
df['Category_LandCapabilityType'].value_counts()

In [ ]:

df = df23
df = pd.melt(df, id_vars=['Majority_CAPABILITY'], value_vars=['Residential_Units', 'TouristAccommodation_Units', 'CommercialFloorArea_SqFt'], var_name='DevelopmentType', value_name='Value')
df = df.groupby(['Majority_CAPABILITY', 'DevelopmentType'], dropna=False).sum().reset_index()
# set Value to int
df['Value'] = df['Value'].astype(int)
df

In [ ]:
# pivot sdfB to get values for Residential_Units by Jurisdiction, Category, and LOCATION_TO_TOWNCENTER
df, sdf = get_summary('2019')
dfCat = pd.melt(sdf, id_vars=['Category','JURISDICTION'], value_vars=['Residential_Units', 'TouristAccommodation_Units', 'CommercialFloorArea_SqFt'], var_name='DevelopmentType', value_name='Value')
dfTC  = pd.melt(sdf, id_vars=['LOCATION_TO_TOWNCENTER','JURISDICTION'], value_vars=['Residential_Units', 'TouristAccommodation_Units', 'CommercialFloorArea_SqFt'], var_name='DevelopmentType', value_name='Value')
# make Category values the column names with totals for each Jurisdiction of Residential_Units, TouristAccommodation_Units, and CommercialFloorArea_SqFt
dfCatPivot = dfCat.pivot_table(index=['JURISDICTION', 'DevelopmentType'], columns='Category', values='Value', aggfunc='sum', fill_value=0).reset_index()
dfTCPivot  = dfTC.pivot_table(index=['JURISDICTION', 'DevelopmentType'], columns='LOCATION_TO_TOWNCENTER', values='Value', aggfunc='sum', fill_value=0).reset_index()

# merge on Jurisdiction and DevelopmentType
dfMerge = pd.merge(dfCatPivot, dfTCPivot, on=['JURISDICTION', 'DevelopmentType'], how='outer', suffixes=('_Category', '_TownCenter'))

# format dfMerge values
# Rename Fields
df = dfMerge.rename(columns={'JURISDICTION': 'Jurisdiction', 
                                  'DevelopmentType': 'Development Type', 
                                  'Further than Quarter Mile from Town Center':'Remote Areas',
                                  "Within Town Center": "Town Centers"})
# format Development Type values
devType = {'CommercialFloorArea_SqFt': 'Commercial Floor Area',
           'Residential_Units': 'Residential Units',
           'TouristAccommodation_Units': 'Tourist Accommodation Units'}
# jursidiction values
jurisdiction = {'CC': 'Carson City County',
                'CSLT': 'City of South Lake Tahoe',
                'EL': 'El Dorado County',
                'DG': 'Douglas County',
                'PL': 'Placer County',
                'WA': 'Washoe County'}
# map values
df['Development Type'] = df['Development Type'].map(devType)
df['Jurisdiction'] = df['Jurisdiction'].map(jurisdiction)
# use gropuby to get total Residential_Units by Jurisdiction
dfRes = sdf.groupby(['JURISDICTION'], dropna=False)['Residential_Units'].sum().reset_index()
dfTAU = sdf.groupby(['JURISDICTION'], dropna=False)['TouristAccommodation_Units'].sum().reset_index()
dfCFA = sdf.groupby(['JURISDICTION'], dropna=False)['CommercialFloorArea_SqFt'].sum().reset_index()
# create Development Type column for each dataframe
dfRes['Development Type'] = 'Residential Units'
dfTAU['Development Type'] = 'Tourist Accommodation Units'
dfCFA['Development Type'] = 'Commercial Floor Area'
# rename to Total
dfRes = dfRes.rename(columns={'Residential_Units': 'Total Existing'})
dfTAU = dfTAU.rename(columns={'TouristAccommodation_Units': 'Total Existing'})
dfCFA = dfCFA.rename(columns={'CommercialFloorArea_SqFt': 'Total Existing'})
# merge dataframes
dfUnits = pd.concat([dfRes, dfTAU, dfCFA])
# map jurisdiction values
dfUnits['JURISDICTION'] = dfUnits['JURISDICTION'].map(jurisdiction)
# rename Jurisdiction column
dfUnits = dfUnits.rename(columns={'JURISDICTION': 'Jurisdiction'})
# sort values
dfUnits = dfUnits.sort_values(by=['Jurisdiction', 'Development Type'], ascending=[True, True], ignore_index=True)

# add Total Existing to dfMerge from dfUnits
df = pd.merge(df, dfUnits, on=['Jurisdiction', 'Development Type'], how='outer', suffixes=('_Category', '_TownCenter'))
df
# drop column
df = df.drop(columns=[''])
df.to_pickle(localpath / 'data' / 'development_summary_by_landcap_landuse_19.pkl')
df.to_csv(localpath / 'data' / 'development_summary_by_landcap_landuse_19.csv', index=False)

In [ ]:
df

## Air Quality

In [ ]:
# get PM 2.5
df = get_airquality_data_sql()
# plot PM 2.5
plot_pm2_5_annual(df, draft=False)

In [ ]:
# get PM 2.5
df = get_airquality_data_sql()
# plot PM 2.5
plot_pm2_5_24hour(df, draft=False)

In [ ]:
# get PM 10
df = get_airquality_data_sql()
#Get PM 10 24 hour high
plot_pm10_24hr(df, draft= False)

In [ ]:
# get PM 10
df = get_airquality_data_sql()
#Plot PM 10 Average Annual Concentrations
plot_pm10_annual(df, draft= False)

In [ ]:
# get 03
df = get_airquality_data_sql()
#Plot 03 1 hour high
plot_o3_1hour_high(df, draft= False)

In [ ]:
# get CO
df = get_airquality_data_sql()
#Plot CO 1 hour high
plot_co_8hour_avg(df, draft= False)

In [ ]:
#Get 50th regionalVisibility Bliss 
df = get_airquality_data_sql()
#Plot Visibility at Bliss
plot_50_Bliss_vis(df, draft= False)

In [ ]:
#Get 90th regionalVisibility Bliss 
df = get_airquality_data_sql()
#Plot 90th percentile Visibility at Bliss
plot_90_Bliss_vis(df, draft= False)

In [ ]:
#Get 50th Visibility subregional slt
df = get_airquality_data_sql()
#Plot 50th Visibility subregional slt
plot_50_SLT_vis(df, draft= False)

In [ ]:
#Get 90th Visibility subregional slt
df = get_airquality_data_sql()
#Plot 90th Visibility subregional slt
plot_90_SLT_vis(df, draft= False)

In [ ]:
#Get NOx data
df = get_NOX_data_sql()
#Plot NOX
plot_NOx(df, draft= False)

In [ ]:
#Get winter traffic data
df = get_wintertraffic_data_sql()
plot_winter_traffic(df, draft= False)

In [ ]:
# mid lake dissolved nitrogen data and chart
df = get_midlake_dissolved_nitrogen_data()
plot_midlake_dissolved_nitrogen(df, draft=False)

## Fisheries

In [ ]:
from fisheries import *

In [ ]:
#Fish habitat
df = get_fishhab()
plot_fishhab(df, draft=False)

In [ ]:
#Stream Habitat Condition
df= get_CSCI()
plot_avgCSCI(df,draft=False)

## Noise

In [ ]:
from noise import *

In [ ]:
# plot commercial noise
# get data
df = get_plannoise_data_sql()
# plot data
plot_commercial(df, draft=False)

In [ ]:
# plot hotel/motel noise data
# get data
df = get_plannoise_data_sql()
# plot data
plot_hotelmotel(df, draft=False)

In [ ]:
# high density residential noise data
# get data
df = get_plannoise_data_sql()
# plot data
plot_highdensityresidential(df, draft=False)

In [ ]:
# plot industrial noise data
# get data
df = get_plannoise_data_sql()
# plot data
plot_industrial(df, draft=False)

In [ ]:
# plot low density residential noise data
# get data
df = get_plannoise_data_sql()
# plot data
plot_lowdensity(df, draft=False)

In [ ]:
# plot rural noise data
# get data
df = get_plannoise_data_sql()
# plot data
plot_rural(df, draft=False)

In [ ]:
# plot urban outdoor recreation noise
# get data
df = get_plannoise_data_sql()
# plot data
plot_rec(df, draft=False)

In [ ]:
# plot wilderness and roadless area noise
# get data
df = get_plannoise_data_sql()
# plot data
plot_wilderness(df, draft=False)

In [ ]:
# plot watercraft shore noise
# get data
df = get_shorenoise_data_sql()
# plot data
plot_watercraft(df, draft=False)

In [ ]:
#plot critical wildlife
df=get_plannoise_data_sql()
#get data
plot_wildlife(df, draft=False)

## Scenic Resources

In [ ]:
# plot scenic corridor roadway attainment
# get data
df = get_scenic_corridor_rating()
# plot data
plot_scenic_corridor_attainment_roadway(df, draft=False)

In [ ]:
# plot scenic corridor shoreline attainment
# get data
df = get_scenic_corridor_rating()
# plot data
plot_scenic_corridor_attainment_shoreline(df, draft=False)

In [ ]:
# plot scenic viewpoint average rating
# get data
df = get_scenic_viewpoint_rating()
# plot data
plot_scenic_viewpoint_roadway_rating(df, draft=True)

In [ ]:
# plot scenic viewpoint shoreline average rating
# get data
df = get_scenic_viewpoint_rating()
# plot data
plot_scenic_viewpoint_shoreline_rating(df, draft=True)

## Soil Conservation

In [ ]:
dfImpChg, dfImp2019 = get_soil_conservation_data_sql()
dfImpOld = transform_soil_conservation_data(dfImpChg, dfImp2019)
dfImpNew = transform_new_coverage_data()
df = add_new_coverage(dfImpOld, dfImpNew)

In [ ]:
# plot 1A - soil conservation
plot_soil_conservation(df, landcap = '1A', draft=False)
plot_soil_conservation(df, landcap = '1B', draft=False)
plot_soil_conservation(df, landcap = '1C', draft=False)
plot_soil_conservation(df, landcap = '2', draft=False)
plot_soil_conservation(df, landcap = '3', draft=False)
plot_soil_conservation(df, landcap = '4', draft=False)
plot_soil_conservation(df, landcap = '5', draft=False)
plot_soil_conservation(df, landcap = '6', draft=False)
plot_soil_conservation(df, landcap = '7', draft=False)

In [ ]:
from soil_conservation import *

In [ ]:
# Get SEZ Data
df = get_sez_data_sql()
# Make SEZ/Wetland Restoration Chart
plot_BasinwideSEZ_scores(df, draft=False) 

## Vegetation

In [ ]:
# Vegetation Preservation Abundance
# get veg data
df =get_ecobject_caldor_identity_data()
# plot veg data
plot_veg_abundance(df, draft=True)

In [ ]:
# Vegetation Seral Stage
# get veg data
df =get_ecobject_caldor_identity_data()
# plot veg data
plot_seral_stage(df, draft=False)

In [ ]:
# Vegetatoin Composition
# get veg data
df =get_ecobject_caldor_identity_data()
# plot veg data
plot_veg_composition(df, draft=True)

In [ ]:
#Get Sensitive plant data
#TDraba, CDraba, Lewisia = get_sensitiveplant_xlsx()
df= get_TDraba_xslx()
#plot_TDraba
plot_TDraba(df, draft=False)

In [ ]:
#Get Cup Draba subpopulation data
df= get_CDraba_xslx()
#plot CDraba
plot_CDraba(df, draft=False)

In [ ]:
#Get Long Petaled Lewisia data
df= get_lewisia_xslx()
#plot CDraba
plot_lewisia(df, draft=False)

In [ ]:
# Relative Abundance of Red Fir Forest In Seral Stages Other Than Mature
# get data
df = get_ecobject_caldor_identity_data()
# plot red fir forest chart
plot_redfir(df, draft=False)

In [ ]:
# Relative Abundance of Yellow Pine Forest In Seral Stages Other Than Mature
# get data
df = get_ecobject_caldor_identity_data()
# plot red fir forest chart
plot_yellowpine(df, draft=False)

In [ ]:
# Relative Abundance of Meadows And Wetland Vegetation Types
# get data
df = get_ecobject_caldor_identity_data()
# plot
plot_wetland(df, draft=False)

In [ ]:
# Relative Abundance of Shrub Vegetation Type
# get data
df = get_ecobject_caldor_identity_data()
# plot
plot_shrub(df, draft=False)

In [ ]:
# Relative Abundance of Deciduous Riparian Vegetation
# get data
df = get_ecobject_caldor_identity_data()
# plot
plot_deciduous(df, draft=True)

In [ ]:
# Pattern: Stand Composition And Age
# get data
df = get_ecobject_caldor_identity_data()
# plot red fir forest chart
plot_veg_composition(df, draft=True)

In [ ]:
# Consistency with Bailey Land Capability System
# get data
dfImpChg, dfImp2019 = get_soil_conservation_data_sql()
# plot land capability chart
plot_landcapability(dfImpChg, draft=False)

In [ ]:
# Appropriate Management Practices
df = get_data_forest_fuel()
plot_forest_fuel(df, draft=True)

In [ ]:
# Total Old Growth
df = get_old_growth()
plot_old_growth(df, draft=False)

In [ ]:
# Old Growth in Sub-Alpine Zone
df = get_ecobject_caldor_identity_data()
plot_seral_subalpine(df, draft=False)

In [ ]:
# Old Growth in Upper Montane Zone
df = get_ecobject_caldor_identity_data()
plot_seral_upper_montane(df, draft=False)

In [ ]:
# Old Growth in Montane Zone
df = get_ecobject_caldor_identity_data()
plot_seral_montane(df, draft=False)

In [ ]:
#Tahoe Yellow Cress Chart
df = get_TYC_data_sql()
plot_TYC(df, draft=False)

## Water Quality

In [ ]:
# secchi depth data and chart
df = get_secchi_data_sql()
# df = get_secchi_data_web()
plot_secchi_depth(df, draft=False)

In [ ]:
# watercraft inspections
df = get_watercraft_inspection_data_web()
plot_watercraft_inspections(df, draft=False)

In [ ]:
from water_quality import *

In [ ]:
# get water quality data
df = get_waterquality_data()
# plot total nitrogetn
plot_total_nitrogen(df, draft=False)
plot_total_phosphorus(df, draft=False)
plot_suspended_sediment(df, draft=False)

In [ ]:
# get total load reduction data
df = get_phosphorus_load_reduction()
plot_phosphorus_load_reduction(df, draft=False)

In [ ]:
# get total load reduction data
df = get_nitrogen_load_reduction()
plot_nitrogen_load_reduction(df, draft=False)

In [ ]:
# get total load reduction data
df = get_sediment_load_reduction()
plot_sediment_load_reduction(df, draft=False)

In [ ]:
# periyphyton data and chart
df = get_periphyton_data()
plot_periphyton(df, draft=False)

In [ ]:
# nearshore turbidity data and chart
df = get_nearshore_turbidity_data()
plot_nearshore_turbidity(df, draft=True)

In [ ]:
df = get_ais_infestation_data_sql()
plot_ais_infestation(df, draft=True)

## Wildlife

In [ ]:
from wildlife import *

In [ ]:
# goshawk data and chart
df = get_wildlife_data_web()
plot_goshawk_data(df, draft=False)

In [ ]:
# osprey data and chart
df = get_wildlife_data_web()
plot_osprey_data(df, draft=False)

In [ ]:
# bald eagle winter roost data and chart
df = get_wildlife_data_web()
plot_bald_eagle_winter(df, draft=False)

In [ ]:
df = get_bald_eagle_data_wt_estimate_sql()
plot_bald_eagle_winter_est(df, draft=False)

In [ ]:
df = get_peregrine_falcon_data_wt_estimate_sql()
plot_pfalcon_data_wt_estimate(df, draft=False)

In [ ]:
df = get_osprey_data_wt_estimate_sql()
plot_osprey_data_wt_estimate(df, draft=False)

In [ ]:
# peregine falcon data and chart
df = get_wildlife_data_web()
plot_falcon_data(df, draft=False)

In [ ]:
# bald eagle summer roost data and chart
df = get_wildlife_data_web()
plot_bald_eagle_summer(df, draft=False)

In [ ]:
#waterfowl data and chart
df = get_waterfowl_data_web()
plot_waterfowl_data(df, draft=False)

# Executive Summary

In [15]:
from executive_summary import *
localpath = pathlib.Path.cwd()
# Summarize Air Quality Data and Relate it to Fires
df = get_airquality_data_sql()
plot_AQ_daily_exceedances_per_year(df, draft=True)

       Year                            Indicator  Average Daily Exceedances
0    1983.0  3-year mean (90th Percentile, Mm-1)                        NaN
1    1984.0  3-year mean (90th Percentile, Mm-1)                        NaN
2    1985.0  3-year mean (90th Percentile, Mm-1)                        NaN
3    1986.0  3-year mean (90th Percentile, Mm-1)                        NaN
4    1987.0  3-year mean (90th Percentile, Mm-1)                        NaN
5    1988.0  3-year mean (90th Percentile, Mm-1)                        NaN
6    1989.0  3-year mean (90th Percentile, Mm-1)                        NaN
7    1990.0  3-year mean (90th Percentile, Mm-1)                        NaN
8    1991.0  3-year mean (90th Percentile, Mm-1)                        NaN
9    1992.0  3-year mean (90th Percentile, Mm-1)                        NaN
10   1993.0  3-year mean (90th Percentile, Mm-1)                        NaN
11   1994.0  3-year mean (90th Percentile, Mm-1)                        NaN
12   1995.0 

In [ ]:
print(out_chart)